# SMILE!!!
## Computational Methods in Psychology and Neuroscience
### Psychology 4215/7215 --- Fall 2023
By: Per B. Sederberg, PhD



# Lesson Objectives

Upon completion of this lesson, students should have learned:

1. To define a hierarchical state machine
2. The difference between build-time and run-time in SMILE
3. The difference between Action and Flow states in SMILE
4. How to build simple experiments in SMILE


# What is SMILE?

- ***State Machine Interface Library for Experiments***
- Goals in developing SMILE:
  - Have millisecond accuracy in timing without difficult code
  - Write experiments that run cross-platform
  - Make easy tasks easy and hard tasks possible
  - Log everything, so you can recreate any experiment

Instead of *coding* you're *smiling*!!!


# Installing SMILE

- First you need Kivy, which is the primary dependency of SMILE
- Sometimes kivy has conflicts with some other packages, so we create a new virtual environment for working with SMILE:

```bash
conda create -n smile
conda activate smile
```

- Now we can install kivy:

```bash
conda install -c conda-forge kivy
```

  - If you plan on running SMILE from within Jupyter Notebook (only suggested for development and not data collection), you'll need to install that, too:

```bash
conda install notebook
```

- Then you can install SMILE right from the GitHub repository:

```bash
pip install git+https://github.com/compmem/smile
```

# Kivy 

- [Kivy](https://kivy.org) is a cross-platform python application development library
- All core libraries are compiled to C code, so it's very fast
- It's built on OpenGL, so it can have powerful graphics
- Possible to deploy your apps on Android, iOS, Windows, OSX, and Linux from one Python code-base.



# What is a State Machine?

- We're really talking about a *finite* state machine, because it does not have unlimited states.
- Are a common way of modeling systems in many fields/areas
- Often represented by a directed graph with nodes as states and edges as transitions:
 
 ![Stoplight](https://miro.medium.com/max/700/1*t03qBGv_S_Iz7dxnOnwt_w.png)

# Hierarchical State Machines

- A very powerful extension of a base state machine is to make it hierarchical
- This just means that states can be entire finite state machines!
- HSMs can represent almost any computer program
  - e.g., most computer games are just really big and complex HSMs

# SMILE helps you build state machines



In [2]:
# load in smile states
from smile.common import *

# create an experiment instance
exp = Experiment(show_splash=False, fullscreen=False)

# show some text for 3 seconds
Label(text="Hello, World!", duration=3)

# run the experiment
exp.run()

# Running a SMILE experiment

- While it's possible to start an experiment inside a notebook, typically you'll start experiments from the command line:

```bash
python exp_name.py -s subj001
```

- The `-s` option allows you to specify a subject id, which will determine where the data are saved.
- There are other command line options, such as `-f` to turn off fullscreen mode:

```bash
python exp_name.py -s subj001 -f
```


# Build-time vs. Run-time

- The most important concept to learn with SMILE is the distinction between *building* a state machine and *running* a state machine.
- During build-time:
  - Calls to the SMILE states construct the state machine
  - Actual values in Python variables will not be available, yet
- During run-time:
  - The state machine is initialized at the first state and runs to completion
  - ***Python code in your script is not run, just the state machine you have constructed.***

# References

- Since you can't evaluate python variables during build time, you need delay evaluations until later.
- References help make that happen:

In [2]:
a = Ref.object(3)
b = Ref.object(4)
c = a + b
print(c.__repr__())
print(c.eval())

Ref(<built-in function add>, Ref(<function pass_thru at 0x7f68d806ecb0>, 3), Ref(<function pass_thru at 0x7f68d806ecb0>, 4))
7


- All state attributes in SMILE are references
  - Meaning you can refer to them at build time and evaluate them at run time

# Action vs. Flow

- Another key concept in SMILE is the distinction between `Action` states and `Flow` states.
- Action states carry out some specific input or output operation and often have a `duration`.
- Flow states control the order of operations for the action states and rarely have a `duration`.

# Action state examples

- `Image`: Presents an image on the screen
- `Label`: Places text on the screen
- `KeyPress`: Accepts specific user input
- `MovingDot`: Present a moving dot stimulus on the screen

# Flow state examples

Most Flow states are parents to other states:

- `Parallel` and `Serial`: Control sequences of states
- `If`, `Elif`, `Else`: Condition branching
- `Loop`: Provide looping over states (optionally with conditionals)
- `Meanwhile`, `UntilDone`: Run some states while others while other states are running
  - These are basically convenience methods for common uses of Parallel.

# Let's build a stop light!

In [3]:
from smile.common import * 

# set up times for each light
red_time = 1.0
yellow_time = 0.5
green_time = 1.5

# define the colors (RGBA)
green_on = [0,1,0,1]
green_off = [0,1,0,.1]
red_on = [1,0,0,1]
red_off = [1,0,0,.1]
yellow_on = [1,1,0,1]
yellow_off = [1,1,0,.1]

radius_prop = 1/6.

num_loops = 2

# make a stoplight exp
exp = Experiment(show_splash=False, fullscreen=False, debug=True)

Wait(1.0)

# put up rectangle with three colored circles (low alpha for off)
with Parallel():
    box = Rectangle(height=300, width=100, color='gray')
    yellow_light = Ellipse(color=yellow_off,
                           radius=box.height*radius_prop)
    red_light = Ellipse(color=red_off, 
                        radius=box.height*radius_prop,
                        bottom=yellow_light.top)
    green_light = Ellipse(color=green_off, 
                          radius=box.height*radius_prop,
                          top=yellow_light.bottom)
    
    # add some labels for the lights
    Label(text='GO', color='black', center=green_light.center)
    Label(text='Wait', color='black', center=yellow_light.center)
    Label(text='STOP', color='black', center=red_light.center)
with UntilDone():
    Wait(until=box.appear_time)
    with Loop(num_loops) as loop:
        # make green light active
        UpdateWidget(green_light, color=green_on)
        Wait(green_time)
        UpdateWidget(green_light, color=green_off)
        
        # make yellow light active
        UpdateWidget(yellow_light, color=yellow_on)
        Wait(yellow_time)
        UpdateWidget(yellow_light, color=yellow_off)
        
        # make red light active
        UpdateWidget(red_light, color=red_on)
        Wait(red_time)
        UpdateWidget(red_light, color=red_off)

Wait(1.0)

# run the experiment
exp.run()


# Let's learn by building together!

- Last class we wrote a list generation for a Flanker task.
- Let's write the frontend experiment to loop over those trials.

# List Gen Function

In [4]:
import random 
import copy

# define the conditions
conds = [{'condition': 'congruent',
          'direction': 'left',
          'stimulus': '<<<<<<<'
         },
         {'condition': 'congruent',
          'direction': 'right',
          'stimulus': '>>>>>>>'
         },
         {'condition': 'incongruent',
          'direction': 'left',
          'stimulus': '>>><>>>'
         },
         {'condition': 'incongruent',
          'direction': 'right',
          'stimulus': '<<<><<<'
         },
         {'condition': 'neutral',
          'direction': 'left',
          'stimulus': '===<==='
         },
         {'condition': 'neutral',
          'direction': 'right',
          'stimulus': '===>==='
         },]

# specify number of reps of these conditions
num_reps = 2

# loop and create the list
trials = []
for i in range(num_reps):
    # extend the trials with copies of the conditions
    trials.extend(copy.deepcopy(conds))

# shuffle the trials
random.shuffle(trials)

print(trials)

[{'condition': 'incongruent', 'direction': 'right', 'stimulus': '<<<><<<'}, {'condition': 'congruent', 'direction': 'right', 'stimulus': '>>>>>>>'}, {'condition': 'incongruent', 'direction': 'left', 'stimulus': '>>><>>>'}, {'condition': 'incongruent', 'direction': 'right', 'stimulus': '<<<><<<'}, {'condition': 'congruent', 'direction': 'left', 'stimulus': '<<<<<<<'}, {'condition': 'neutral', 'direction': 'left', 'stimulus': '===<==='}, {'condition': 'congruent', 'direction': 'right', 'stimulus': '>>>>>>>'}, {'condition': 'neutral', 'direction': 'right', 'stimulus': '===>==='}, {'condition': 'incongruent', 'direction': 'left', 'stimulus': '>>><>>>'}, {'condition': 'neutral', 'direction': 'right', 'stimulus': '===>==='}, {'condition': 'neutral', 'direction': 'left', 'stimulus': '===<==='}, {'condition': 'congruent', 'direction': 'left', 'stimulus': '<<<<<<<'}]


# Goal for each trial

- Present the correct stimulus as text on the screen
- Wait for a response
- Remove the stimulus
- Wait for an inter-stimulus interval
- Log the result of the trial

In [5]:
from smile.common import *

font_size = 75
resp_keys = ['F', 'J']
ISI_dur = 0.5
ISI_jitter = 0.5

# create the experiment
exp = Experiment(show_splash=False, fullscreen=False)

# show the stimulus (will default to center of the screen)
with Loop(trials) as trial:
    stim = Label(text=trial.current['stimulus'],
                 font_size=font_size)
    with UntilDone():
        kp = KeyPress(keys=resp_keys)
    
    Wait(ISI_dur, jitter=ISI_jitter)
    
    Log(trial.current, name='flanker',
        stim_on=stim.appear_time,
        resp=kp.pressed,
        resp_time=kp.press_time
       )
    
# run the experiment
exp.run()

## Tips

- When in doubt, draw it out!
  - Since SMILE just creates state machines, you can draw out exactly the flow of actions and that can help you translate it into code.
- Debugging is hard in a state machine, so make use of the `Debug` state to help you by printing out various values at run time.

- The SMILE docs need updating, but go into more detail: https://smile-docs.readthedocs.io/en/latest/index.html

In [16]:
# Moving Dot Example
from smile.common import *

# set up params
NUM_DOTS = 100
RIGHT_COH = .3
LEFT_COH = .1
RADIUS = 275
LIFESPAN = 0.75
LIFESPAN_VAR = 0.5
SPEED = 200.0
TEST_WIDTH = 1600
COLOR = 'white'
UPDATE_INTERVAL = 1./30.
DURATION = 5.0
INT_TIME = 3.0

# create the top-level experiment
exp = Experiment(show_splash=False, fullscreen=False)

# Wait before starting the dots
Wait(0.5)

# present the moving dot stimulus with desired params
md = MovingDots(color=COLOR, 
                num_dots=NUM_DOTS, radius=RADIUS,
                motion_props=[{"coherence": RIGHT_COH,
                               "direction": 0,
                               "direction_variance": 0},
                              {"coherence": LEFT_COH,
                               "direction": 180,
                               "direction_variance": 0}],
                lifespan=LIFESPAN,
                lifespan_variance=LIFESPAN_VAR,
                speed=SPEED, update_interval=UPDATE_INTERVAL,
                duration=DURATION)
with Meanwhile():
    # while the dots are up
    # wait until they've appeared on the screen
    Wait(until=md.appear_time)
    
    # then leave the dots moving until the interogation time
    Wait(INT_TIME)
    
    # stop updating the dot locations
    md.update(update_interval=10000.)

# run the experiment
exp.run()

## Assignment before next class

- Your first mini list generation code is due next Sunday by midnight!
- Your main listgen code project (A02) will be due in two weeks.
- Start familiarizing yourself with SMILE, since that will be front and center for the next assignment.


### See you next week!!!